In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Load csv and main config
path = Path("/kaggle", "input", "planets-dataset", "planet", "planet", "train-jpg")

train_classes = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/train_classes.csv")
train_classes["tag"] = train_classes["tags"].apply(lambda x: sorted(x.split(" "))[-1])

train_labels = {
    row["image_name"]: row["tag"] for i, row in train_classes.iterrows()
}

In [3]:
# Count classes
train_classes.tag.value_counts(normalize=True)

primary              0.606438
water                0.183083
road                 0.142889
cloudy               0.051607
selective_logging    0.007189
slash_burn           0.004570
clear                0.003137
partly_cloudy        0.000568
haze                 0.000296
habitation           0.000173
cultivation          0.000049
Name: tag, dtype: float64

In [4]:
# Load data
from datasets import load_dataset, Image

dataset = load_dataset("imagefolder", data_dir=path)


Resolving data files:   0%|          | 0/40479 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-e47733c92f89b3ef/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Load feature extractor
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [6]:
# Example of feature extractor
image = dataset["train"][300]["image"].convert("RGB")
feature_extractor(image, return_tensors='pt')


{'pixel_values': tensor([[[[-0.4510, -0.4510, -0.4745,  ..., -0.4902, -0.4824, -0.4667],
          [-0.4510, -0.4353, -0.4588,  ..., -0.4980, -0.4824, -0.4667],
          [-0.4824, -0.4588, -0.4824,  ..., -0.5137, -0.4980, -0.4667],
          ...,
          [-0.5373, -0.5216, -0.5059,  ..., -0.5294, -0.5294, -0.5294],
          [-0.5059, -0.5059, -0.4980,  ..., -0.5373, -0.5451, -0.5451],
          [-0.4824, -0.4824, -0.4980,  ..., -0.5294, -0.5373, -0.5294]],

         [[-0.4510, -0.4431, -0.4588,  ..., -0.4588, -0.4510, -0.4353],
          [-0.4431, -0.4275, -0.4588,  ..., -0.4667, -0.4588, -0.4431],
          [-0.4510, -0.4275, -0.4510,  ..., -0.4824, -0.4667, -0.4431],
          ...,
          [-0.4980, -0.4745, -0.4588,  ..., -0.4980, -0.4980, -0.4902],
          [-0.4745, -0.4667, -0.4588,  ..., -0.5059, -0.5137, -0.5137],
          [-0.4510, -0.4510, -0.4588,  ..., -0.4980, -0.5216, -0.5294]],

         [[-0.5216, -0.4980, -0.5137,  ..., -0.5294, -0.5294, -0.5216],
          [-0

In [7]:
# Process 1 example and process batch methods

def process_example(example):
    inputs = feature_extractor(example['image'].convert("RGB"), return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert("RGB") for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

In [8]:
# Process all dataset
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']
prepared_ds_train = train_ds.with_transform(transform)
prepared_ds_val = val_ds.with_transform(transform)

In [9]:
# Process 1 example
process_example(dataset['train'][0])



{'pixel_values': tensor([[[[-0.2471, -0.2627, -0.2549,  ..., -0.2549, -0.2392, -0.2549],
          [-0.2706, -0.2863, -0.2706,  ..., -0.2706, -0.2627, -0.2706],
          [-0.2941, -0.3020, -0.2784,  ..., -0.2863, -0.2863, -0.2941],
          ...,
          [-0.2706, -0.2627, -0.2627,  ..., -0.3176, -0.3098, -0.3490],
          [-0.2549, -0.2549, -0.2627,  ..., -0.2941, -0.3020, -0.3412],
          [-0.2314, -0.2549, -0.2627,  ..., -0.2706, -0.2863, -0.3412]],

         [[-0.1294, -0.1294, -0.1137,  ..., -0.1294, -0.1294, -0.1451],
          [-0.1529, -0.1529, -0.1373,  ..., -0.1529, -0.1529, -0.1529],
          [-0.1686, -0.1608, -0.1529,  ..., -0.1686, -0.1608, -0.1608],
          ...,
          [-0.1294, -0.1294, -0.1216,  ..., -0.1451, -0.1529, -0.1922],
          [-0.1373, -0.1373, -0.1294,  ..., -0.1373, -0.1451, -0.1843],
          [-0.1529, -0.1529, -0.1529,  ..., -0.1373, -0.1451, -0.1922]],

         [[-0.1843, -0.2000, -0.2000,  ..., -0.1922, -0.1922, -0.1922],
          [-0

In [10]:
# Collation: not sure what it does
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [11]:
# Load accuracy metric: not sure how it works
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


In [12]:
# Dataset label looks like crap
dataset["train"].features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(num_classes=1, names=['train-jpg'], id=None)}

In [13]:
# Prep model
from transformers import ViTForImageClassification

# Prep labels
id2label = train_labels
label2id = {v: k for k, v in id2label.items()}
labels = list(id2label.items())

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=1,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)


In [15]:
prepared_ds_train
prepared_ds_val

Dataset({
    features: ['image', 'label'],
    num_rows: 4048
})

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds_train,
    eval_dataset=prepared_ds_val,
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [17]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36431
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1139


Step,Training Loss,Validation Loss,Accuracy
100,1.521200,1.162683,1.000000
200,0.010400,0.009681,1.000000
300,0.004500,0.004320,1.000000
400,0.002800,0.002687,1.000000
500,0.002000,0.001928,1.000000
600,0.001500,0.001513,1.000000
700,0.001300,0.001261,1.000000
800,0.001100,0.001095,1.000000
900,0.001000,0.000993,1.000000
1000,0.000900,0.000925,1.000000


***** Running Evaluation *****
  Num examples = 4048
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-100
Configuration saved in ./vit-base-beans/checkpoint-100/config.json
Model weights saved in ./vit-base-beans/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./vit-base-beans/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 4048
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-200
Configuration saved in ./vit-base-beans/checkpoint-200/config.json
Model weights saved in ./vit-base-beans/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./vit-base-beans/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 4048
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-300
Configuration saved in ./vit-base-beans/checkpoint-300/config.json
Model weights saved in ./vit-base-beans/checkpoint-300/pytorch_model.bin
Feature extractor saved i

TrainOutput(global_step=1139, training_loss=0.4422884862346671, metrics={'train_runtime': 1540.0779, 'train_samples_per_second': 23.655, 'train_steps_per_second': 0.74, 'total_flos': 3.84728595718041e+18, 'train_loss': 0.4422884862346671, 'epoch': 1.0})

In [18]:
output_path = Path("/kaggle") / "working"

trainer.save_model(output_path)
feature_extractor.save_pretrained(output_path)


Saving model checkpoint to /kaggle/working
Configuration saved in /kaggle/working/config.json
Model weights saved in /kaggle/working/pytorch_model.bin
Feature extractor saved in /kaggle/working/preprocessor_config.json
Feature extractor saved in /kaggle/working/preprocessor_config.json


['/kaggle/working/preprocessor_config.json']